In [1]:
import sys
sys.path.append('D:\\Compute Science\\Machine Learning\\论文\\项目\\FairSPL\\venv_torch')
sys.path.append('D:\\Compute Science\\Machine Learning\\论文\\项目\\FairSPL\\venv_torch\\lib\\site-packages')

In [2]:
import torch
from torch import nn
import numpy as np

In [3]:
# 定义一个简单的模型
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(4, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)

        return x
    
loss_fn = nn.MSELoss()

In [33]:
# 输出模型的参数
model = MLP()
for param in model.parameters():
    print(param.size())

torch.Size([3, 4])
torch.Size([3])
torch.Size([2, 3])
torch.Size([2])


In [34]:
# 定义数据
data = torch.tensor([1,2,3,4], dtype=torch.float)
label = torch.tensor([5,6], dtype=torch.float)
pred = model(data)
loss = loss_fn(pred, label)

计算雅克比矩阵（一阶导数）

In [38]:
def grad(model, y):
    """ 计算一阶导数.
    Returns: 
        grads, grads[i]: dy / dx_i
    """
    model.eval()
    grads = torch.autograd.grad(loss, model.parameters(), retain_graph=True, create_graph=True)
    # for grad in grads:
    #     print(grad.size()) # 可以发现一共 4 个 Tensor，分别为损失函数对四个参数 Tensor（两层，每层都有权重和偏置）的梯度。
        
    grads = torch.cat([x.flatten() for x in grads], dim=0)
    return grads

grads = grad(model, loss)
print(grads.size())

tensor([ -1.8790,  -3.7581,  -5.6371,  -7.5161,   1.2390,   2.4781,   3.7171,
          4.9561,  -1.3145,  -2.6290,  -3.9435,  -5.2579,  -1.8790,   1.2390,
         -1.3145,   7.3481,  -4.0111,  -9.5360,   9.2882,  -5.0701, -12.0537,
         -5.1941,  -6.5655], grad_fn=<CatBackward0>)


计算 Hessian 矩阵（二阶导数）

In [7]:
# 如果函数定义为 ``hess(w, y, ...)
# 直接传入 model.parameters()，会报错，
# 与grad_z唯一的不同是使用了两次 w（grad_z可以这么定义，直接传入model.parameters()）
# 目前不知道原因
def hess(model, y, grads=None):
    """ 计算二阶导数.
    Returns: 
        he, he[i,j]: d^2y / (dx_i dx_j)
    """
    if grads is None:
        grads = grad(model, y)
       
    model.eval()
    total_params = sum(p.numel() for p in model.parameters())
    he = torch.zeros(total_params, total_params)
    
    for i, g in enumerate(grads):
        second_order_grad = grad(model, g)
        he[i, :] = second_order_grad

    return he

he = hess(model, loss)
print(he.size())

torch.Size([23, 23])


In [8]:
# # 计算 hessian 矩阵
# grad = torch.autograd.grad(outputs=loss, inputs=model.parameters(), create_graph=True)
# grad = torch.cat([x.flatten() for x in grad], dim=0)
# total_params = sum(p.numel() for p in model.parameters())
# he2 = torch.zeros(total_params, total_params)
    
# for i, g in enumerate(grad):
#     second_order_grad = torch.autograd.grad(outputs=g, inputs=model.parameters(), retain_graph=True)
#     second_order_grad = torch.cat([x.flatten() for x in second_order_grad], dim=0)
#     he2[i, :] = second_order_grad

实现影响函数（一次性返回所有点的影响函数值）
$$
\mathcal{I}_{\text {up,loss }}\left(z, z_{\text {test }}\right) =-\nabla_{\theta} L\left(z_{\text {test }}, \hat{\theta}\right)^{\top} H_{\hat{\theta}}^{-1} \nabla_{\theta} L(z, \hat{\theta})
$$

In [9]:
# 数据准备
train_z = torch.tensor([[1,2,3,4],[5,6,7,8]], dtype=torch.float)
train_t = torch.tensor([[9,10],[11,12]], dtype=torch.float)

train_set = [(z, t) for z, t in zip(train_z, train_t)]

test_z = torch.tensor([5,6,7,7], dtype=torch.float)
test_t = torch.tensor([7,8], dtype=torch.float)

model = MLP()

In [10]:
# Step1: 计算V第三项
def grad_z(model, z, t):
    """ Calculates the gradient z. One grad_z should be computed for each
    training sample.
    
    Arguments:
        z: torch tensor, training data points
            e.g. an image sample (batch_size, 3, 256, 256)
        t: torch tensor, training data labels
        model: torch NN, model used to evaluate the dataset
    Returns:
        grad_z: list of torch tensor, containing the gradients
            from model parameters to loss
    """
    model.eval()
    y = model(z)
    loss = loss_fn(y, t)
    params = [ p for p in model.parameters() if p.requires_grad ]
    grads = list(torch.autograd.grad(loss, params, create_graph=True))
    
    #for grad in grads:
    #   print(grad.size()) # 可以发现一共 4 个 Tensor，分别为损失函数对四个参数 Tensor（两层，每层都有权重和偏置）的梯度。
        
    return grads

grad = grad_z(model, train_z, train_t)
grad


[tensor([[ 34.4961,  45.1405,  55.7848,  66.4292],
         [  8.2716,  10.8364,  13.4013,  15.9661],
         [-25.4239, -33.2688, -41.1137, -48.9586]], grad_fn=<TBackward0>),
 tensor([10.6444,  2.5649, -7.8449], grad_fn=<ViewBackward0>),
 tensor([[-15.6780,  23.3260,   7.7328],
         [-18.5366,  27.5539,   9.1455]], grad_fn=<TBackward0>),
 tensor([-10.3758, -12.2205], grad_fn=<ViewBackward0>)]

In [11]:
# 验证是否是每一个样本的loss的梯度的和的均值等于在batch上loss的计算结果
grad1, grad2 = [grad_z(model, z, t) for (z, t) in train_set]
g1 = torch.cat([x.flatten() for x in grad], dim=0)
g2 = torch.cat([x.flatten() for x in grad1], dim=0)
g3 = torch.cat([x.flatten() for x in grad2], dim=0)
g1.allclose((g2+g3)/2)

True

In [18]:
# Step2: 计算前两项，也就是 s_test=v^T H^{-1}
# 辅助函数：快速计算 Hv，其中 H 是 Hessian 矩阵
def hvp(y, w, v): # 计算 y 对 w 的二阶导 H，返回 Hv
    """Multiply the Hessians of y and w by v.
    Uses a backprop-like approach to compute the product between the Hessian
    and another vector efficiently, which even works for large Hessians.
    Example: if: y = 0.5 * w^T A x then hvp(y, w, v) returns and expression
    which evaluates to the same values as (A + A.t) v.

    Arguments:
        y: scalar/tensor, for example the output of the loss function
        w: list of torch tensors, tensors over which the Hessian
            should be constructed
        v: list of torch tensors, same shape as w,
            will be multiplied with the Hessian

    Returns:
        return_grads: list of torch tensors, contains product of Hessian and v.

    Raises:
        ValueError: `y` and `w` have a different length."""
    if len(w) != len(v):
        raise(ValueError("w and v must have the same length."))

    # First backprop
    first_grads = torch.autograd.grad(y, w, retain_graph=True, create_graph=True)
    # Elementwise products
    elemwise_products = 0
    for grad_elem, v_elem in zip(first_grads, v):
        elemwise_products += torch.sum(grad_elem * v_elem)


    # Second backprop
    return_grads = torch.autograd.grad(elemwise_products, w, create_graph=True)

    return return_grads


def s_test(model, test_z, test_t, train_loader, damp=0.01, scale=25.0,
       recursion_depth=5000):
    """s_test can be precomputed for each test point of interest, and then
    multiplied with grad_z to get the desired value for each training point.
    Here, strochastic estimation is used to calculate s_test. s_test is the
    Inverse Hessian Vector Product.

    Arguments:
        test_z: torch tensor, test data points, such as test images
        test_t: torch tensor, contains all test data labels
        model: torch NN, model used to evaluate the dataset
        train_loader: torch Dataloader, can load the training dataset
        damp: float, dampening factor
        scale: float, scaling factor
        recursion_depth: int, number of iterations aka recursion depth
            should be enough so that the value stabilises.

    Returns:
        h_estimate: list of torch tensors, s_test
    """
    v = grad_z(model, test_z, test_t)
    h_estimate = v.copy()
    for i in range(recursion_depth):
        for z, t in train_loader:
            y = model(z)
            loss = loss_fn(y, t)
            params = [ p for p in model.parameters() if p.requires_grad ]
            hv = hvp(loss, params, h_estimate) 
            h_estimate = [
                _v + (1 - damp) * _h_e - _hv / scale
                for _v, _h_e, _hv in zip(v, h_estimate, hv)]
            break
    return h_estimate

In [19]:
# 因为内层for循环一致算同一个点，所以当迭代次数增多，误差会很大
# 正确的做法是随机从训练集取一定数量的点
s_test(model, test_z, test_t, train_set, recursion_depth=1)

[tensor([[ -81.2594, -125.6682, -170.0769, -205.2730],
         [  45.3421,   64.4313,   83.5204,   96.0463],
         [  48.8245,   76.4019,  103.9794,  126.2451]], grad_fn=<SubBackward0>),
 tensor([-44.4088,  19.0891,  27.5775], grad_fn=<SubBackward0>),
 tensor([[ 437.3736,   44.4160, -313.7211],
         [ 498.6012,   91.0806, -363.9464]], grad_fn=<SubBackward0>),
 tensor([29.4857, 17.3848], grad_fn=<SubBackward0>)]

In [28]:
# Step3: 合并所有的结果 计算每一个样本点对test_z的影响值
def calc_influence_function(model, train_set, test_z, test_t):
    """Calculates the influence function

    Returns:
        influence: list of float, influences of all training data samples
            for one test sample
        harmful: list of float, influences sorted by harmfulness
        helpful: list of float, influences sorted by helpfulness.
    """
    train_dataset_size = len(train_set)
    influences = []
    
    # list of torch tensor, containing the gradients from model parameters to loss
    grad_z_vecs = [grad_z(model, z, t) for z, t in train_set] 
    # list of torch tensor, contains s_test vectors
    e_s_test = s_test(model, test_z, test_t, train_set, recursion_depth=1)
    
    # 对第 i 个样本
    for i in range(train_dataset_size):
        influence_i = -sum(
            [
                torch.sum(k * j).data.cpu().numpy()
                for k, j in zip(grad_z_vecs[i], e_s_test)
            ]) / train_dataset_size
        influences.append(influence_i)

    harmful = np.argsort(influences)
    helpful = harmful[::-1]

    return influences, harmful.tolist(), helpful.tolist()

In [23]:
calc_influence_function(model, train_set, test_z, test_t)

[tensor([[ -81.2594, -125.6682, -170.0769, -205.2730],
        [  45.3421,   64.4313,   83.5204,   96.0463],
        [  48.8245,   76.4019,  103.9794,  126.2451]], grad_fn=<SubBackward0>), tensor([-44.4088,  19.0891,  27.5775], grad_fn=<SubBackward0>), tensor([[ 437.3736,   44.4160, -313.7211],
        [ 498.6012,   91.0806, -363.9464]], grad_fn=<SubBackward0>), tensor([29.4857, 17.3848], grad_fn=<SubBackward0>)]


([14395.082336425781, 47184.150970458984], [0, 1], [1, 0])

In [30]:
# Step1: 计算V第三项
def grad_z2(model, z, t):
    """ Calculates the gradient z. One grad_z should be computed for each
    training sample.
    
    Arguments:
        z: torch tensor, training data points
            e.g. an image sample (batch_size, 3, 256, 256)
        t: torch tensor, training data labels
        model: torch NN, model used to evaluate the dataset
    Returns:
        grad_z: list of torch tensor, containing the gradients
            from model parameters to loss
    """
    model.eval()
    y = model(z)
    loss = loss_fn(y, t)
    grads = torch.autograd.grad(loss, model.parameters(), create_graph=True)
    grads = torch.cat([x.flatten() for x in grads], dim=0)

    return grads

# grad = grad_z2(model, train_z, train_t)
# grad
# Step2: 计算前两项，也就是 s_test=v^T H^{-1}
# 辅助函数：快速计算 Hv，其中 H 是 Hessian 矩阵
def hvp2(y, model, v): # 计算 y 对 w 的二阶导 H，返回 Hv
    """Multiply the Hessians of y and w by v.
    Uses a backprop-like approach to compute the product between the Hessian
    and another vector efficiently, which even works for large Hessians.
    Example: if: y = 0.5 * w^T A x then hvp(y, w, v) returns and expression
    which evaluates to the same values as (A + A.t) v.

    Arguments:
        y: scalar/tensor, for example the output of the loss function
        w: list of torch tensors, tensors over which the Hessian
            should be constructed
        v: list of torch tensors, same shape as w,
            will be multiplied with the Hessian

    Returns:
        return_grads: list of torch tensors, contains product of Hessian and v.

    """
    # First backprop
    first_grads = torch.autograd.grad(y, model.parameters(), retain_graph=True, create_graph=True)
    first_grads = torch.cat([x.flatten() for x in first_grads], dim=0)
    # Elementwise products
    elemwise_products = 0
    for grad_elem, v_elem in zip(first_grads, v):
        elemwise_products += torch.sum(grad_elem * v_elem)
    
    
    # Second backprop
    return_grads = torch.autograd.grad(elemwise_products, model.parameters(), create_graph=True)
    return_grads = torch.cat([x.flatten() for x in return_grads], dim=0)
    return return_grads


def s_test2(model, test_z, test_t, train_loader, damp=0.01, scale=25.0,
       recursion_depth=5000):
    """s_test can be precomputed for each test point of interest, and then
    multiplied with grad_z to get the desired value for each training point.
    Here, strochastic estimation is used to calculate s_test. s_test is the
    Inverse Hessian Vector Product.

    Arguments:
        test_z: torch tensor, test data points, such as test images
        test_t: torch tensor, contains all test data labels
        model: torch NN, model used to evaluate the dataset
        train_loader: torch Dataloader, can load the training dataset
        damp: float, dampening factor
        scale: float, scaling factor
        recursion_depth: int, number of iterations aka recursion depth
            should be enough so that the value stabilises.

    Returns:
        h_estimate: list of torch tensors, s_test
    """
    v = grad_z2(model, test_z, test_t)
    h_estimate = v.clone() # 不能再 .detache() !!!
    for i in range(recursion_depth):
        for z, t in train_loader:
            y = model(z)
            loss = loss_fn(y, t)
            hv = hvp2(loss, model, h_estimate) 
            h_estimate = [
                _v + (1 - damp) * _h_e - _hv / scale
                for _v, _h_e, _hv in zip(v, h_estimate, hv)]
            break
    return h_estimate

# Step3: 合并所有的结果 计算每一个样本点对test_z的影响值
def calc_influence_function2(model, train_set, test_z, test_t):
    """Calculates the influence function

    Arguments:
        grad_z_vecs: list of torch tensor, containing the gradients
            from model parameters to loss
        e_s_test: list of torch tensor, contains s_test vectors

    Returns:
        influence: list of float, influences of all training data samples
            for one test sample
        harmful: list of float, influences sorted by harmfulness
        helpful: list of float, influences sorted by helpfulness.
    """
    train_dataset_size = len(train_set)
    influences = []
    
    grad_z_vecs = [grad_z2(model, z, t) for z, t in train_set] 
    e_s_test = s_test2(model, test_z, test_t, train_set, recursion_depth=1)
    
    
#     grad_z_vecs = torch.cat([x.flatten() for x in grad_z_vecs], dim=0)
#     e_s_test = torch.cat([x.flatten() for x in e_s_test], dim=0)
    
    # 对第 i 个样本
    for i in range(train_dataset_size):
        influence_i = -sum(
            [
                torch.sum(k * j).data.cpu().numpy()
                for k, j in zip(grad_z_vecs[i], e_s_test)
            ]) / train_dataset_size
        influences.append(influence_i)

    harmful = np.argsort(influences)
    helpful = harmful[::-1]

    return influences, harmful.tolist(), helpful.tolist()

calc_influence_function2(model, train_set, test_z, test_t)

([14395.081895828247, 47184.14908027649], [0, 1], [1, 0])